# Movement annotation IV: Testing the interrater agreement between manual and automatic annotation

In this script, we prepare data to test the interrater agreement on movement annotation. We use EasyDIAG to compute the agreement XXX (REF)


In [96]:
import os
import glob
import numpy as np
import pandas as pd

curfolder = os.getcwd()
print(curfolder)

# where do we store our processed timeseries data
processedfolder = os.path.join(curfolder + '\\..\\03_TS_processing\\TS_merged\\')
print(processedfolder)
processedfiles = glob.glob(processedfolder + '*.csv')
print(processedfiles)

annotatedfolder = os.path.join(curfolder + '\\ML_Annotations\\')
# get all folders in the annotated folder
folders = glob.glob(annotatedfolder + '*\\')

folders60 = [x for x in folders if '0_6' in x] #60percent confidence
folders80 = [x for x in folders if '0_8' in x] #80percent confidence


manualfolder1 = os.path.join(curfolder + '\\ManualAnno\\R1\\')
manualfiles1 = glob.glob(manualfolder1 + '*.eaf')
manualfiles1 = [x for x in manualfiles1 if 'ELAN_tiers' in x]

manualfolder2 = os.path.join(curfolder + '\\ManualAnno\\R3\\')
manualfiles2 = glob.glob(manualfolder2 + '*.eaf')
manualfiles2 = [x for x in manualfiles2 if 'ELAN_tiers' in x]

# get all files in the folder
files = glob.glob(test + '*.csv')
print(files)

# here we will store the output for easyDIAG
interfolder = curfolder + '\\InterAg\\'

e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation
e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\..\03_TS_processing\TS_merged\
['e:\\FLESH_ContinuousBodilyEffort\\04_TS_movementAnnotation\\..\\03_TS_processing\\TS_merged\\merged_0_1_10_p1.csv', 'e:\\FLESH_ContinuousBodilyEffort\\04_TS_movementAnnotation\\..\\03_TS_processing\\TS_merged\\merged_0_1_18_p0.csv', 'e:\\FLESH_ContinuousBodilyEffort\\04_TS_movementAnnotation\\..\\03_TS_processing\\TS_merged\\merged_0_1_2_p0.csv', 'e:\\FLESH_ContinuousBodilyEffort\\04_TS_movementAnnotation\\..\\03_TS_processing\\TS_merged\\merged_0_1_8_p0.csv', 'e:\\FLESH_ContinuousBodilyEffort\\04_TS_movementAnnotation\\..\\03_TS_processing\\TS_merged\\merged_0_1_9_p1.csv', 'e:\\FLESH_ContinuousBodilyEffort\\04_TS_movementAnnotation\\..\\03_TS_processing\\TS_merged\\merged_0_2_111_p1.csv', 'e:\\FLESH_ContinuousBodilyEffort\\04_TS_movementAnnotation\\..\\03_TS_processing\\TS_merged\\merged_0_2_112_p1.csv', 'e:\\FLESH_ContinuousBodilyEffort\\0

In [69]:
manualfiles2

[]

Now we need to get both manual and automatic annotations into format that EasyDIAG requires - so simple .txt files with timestamps and annotation values

In [72]:
import xml.etree.ElementTree as ET

def parse_eaf_file(eaf_file, rel_tiers):
    tree = ET.parse(eaf_file)
    root = tree.getroot()

    time_order = root.find('TIME_ORDER')
    time_slots = {time_slot.attrib['TIME_SLOT_ID']: time_slot.attrib['TIME_VALUE'] for time_slot in time_order}

    annotations = []
    relevant_tiers = {rel_tiers}
    for tier in root.findall('TIER'):
        tier_id = tier.attrib['TIER_ID']
        if tier_id in relevant_tiers:
            for annotation in tier.findall('ANNOTATION/ALIGNABLE_ANNOTATION'):
                # Ensure required attributes are present
                if 'TIME_SLOT_REF1' in annotation.attrib and 'TIME_SLOT_REF2' in annotation.attrib:
                    ts_ref1 = annotation.attrib['TIME_SLOT_REF1']
                    ts_ref2 = annotation.attrib['TIME_SLOT_REF2']
                    # Get annotation ID if it exists, otherwise set to None
                    ann_id = annotation.attrib.get('ANNOTATION_ID', None)
                    annotation_value = annotation.find('ANNOTATION_VALUE').text.strip()
                    annotations.append({
                        'tier_id': tier_id,
                        'annotation_id': ann_id,
                        'start_time': time_slots[ts_ref1],
                        'end_time': time_slots[ts_ref2],
                        'annotation_value': annotation_value
                    })

    return annotations


In [140]:

# define which folder you want to use
foi = manualfiles2
raterIDfile = 'R3'
raterID = 'R2'

# IA files
txtfile_mov = interfolder + raterIDfile + '_Manual_mov.txt'
txtfile_head = interfolder + raterIDfile + '_Manual_head.txt'
txtfile_upper = interfolder + raterIDfile + '_Manual_upper.txt'
txtfile_lower = interfolder + raterIDfile + '_Manual_lower.txt'
txtfile_arms = interfolder + raterIDfile + '_Manual_arms.txt'


with open(txtfile_mov, 'w') as f:
    for file in foi:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        annotator = 'Anno_' + raterID  # no matter what ID, for easydiag it always has to be agreement between R1 and R2
        # parse the file
        annotations = parse_eaf_file(file, 'movement_in_trial')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotator}\t{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")

with open(txtfile_head, 'w') as f:
    for file in foi:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        # parse the file
        annotations = parse_eaf_file(file, 'head_mov')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotator}\t{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")

with open(txtfile_upper, 'w') as f:
    for file in foi:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        annotator = 'Anno_' + raterID
        # parse the file
        annotations = parse_eaf_file(file, 'upper_body')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotator}\t{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")

with open(txtfile_lower, 'w') as f:
    for file in foi:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        annotator = 'Anno_' + raterID
        # parse the file
        annotations = parse_eaf_file(file, 'lower_body')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotator}\t{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")

with open(txtfile_arms, 'w') as f:
    for file in foi:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        annotator = 'Anno_' + raterID
        # parse the file
        annotations = parse_eaf_file(file, 'arms')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotator}\t{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")




working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_11_p1_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_12_p1_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_13_p1_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_14_p1_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_15_p1_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_16_p1_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_17_p1_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_20_p0_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ManualAnno\R3\0_1_21_p0_ELAN_tiers.eaf
working on e:\FLESH_ContinuousBodilyE

Now we create similar files from files that were annotated by the classifier

In [137]:
test = folders80[0]
testfiles = glob.glob(test + '*.csv')   

for file in testfiles:
    print('working on ' + file)
    anno_df = pd.read_csv(file)
    # get the filename
    filename = file.split('\\')[-1].split('.')[0]
    filename = filename.split('_')[2:6]
    filename = '_'.join(filename)
    
    # check in manualfiles1 if the file is there
    manualfile = [x for x in processedfiles if filename in x]

    if len(manualfile) > 0:
        df_manual = pd.read_csv(manualfile[0])
        # check last time of anno_df
        lasttime = anno_df['time_ms'].iloc[-1]
        # check last time of manual file
        lasttime_manual = df_manual['time'].iloc[-1]

        print(lasttime)
        print(lasttime_manual+2)
        print(lasttime - lasttime_manual+2)

working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_11_p1_time_annotations_threshold_0_8.csv
3196
3116.0
84.0
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_12_p1_time_annotations_threshold_0_8.csv
3724
3628.0
100.0
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_13_p1_time_annotations_threshold_0_8.csv
3484
3388.0
100.0
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_14_p1_time_annotations_threshold_0_8.csv
5212
5120.0
96.0
working on e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_15_p1_time_annotations_threshold_0_8.csv
4060
3976.0
88.0
working on e:\FLESH_ContinuousBod

In [130]:
for folder in folders80:
    # get the first element of the folder
    tier = folder.split('\\')[-2].split('_')[0]

    if tier == 'heads':
        tier = 'head'
    elif tier == 'upperBody':
        tier = 'upper'
    elif tier == 'lowerBody':
        tier = 'lower'

    txtfile = interfolder + 'AutoAnno_' + tier + '.txt'

    # get all files in the folder
    files = glob.glob(folder + '*.csv')

    for file in files:
        print(file)

        # get the filename
        filename = file.split('\\')[-1].split('.')[0]
        filename = filename.split('_')[2:6]
        filename = '_'.join(filename)

        # check in manualfiles1 if the file is there
        manualfile = [x for x in manualfiles1 if filename in x]

        # if not, we skip
        if len(manualfile) == 0:
            continue

        # now we get the chunks
        anno_df = pd.read_csv(file)


        #### FIRST WE NEED TO APPLY FEW RULES
        ## if there is short pause - meaning no movement - between two movement chunks, for less than 200ms, we will turn it into movement
        #chunks = []

        # get the chunks in anno_values such that when value changes, we have a new chunk

        #get the chunks in anno_values such that when value changes, we have a new chunk
        anno_df['chunk'] = (anno_df['anno_values'] != anno_df['anno_values'].shift()).cumsum()
        # create column idx to keep track of the index
        anno_df['idx'] = anno_df.index

        # Calculate start and end of each chunk, grouped by anno_values, save also the first and last index
        chunks = anno_df.groupby(['anno_values', 'chunk']).agg(
            time_ms_min=('time_ms', 'first'),
            time_ms_max=('time_ms', 'last'),
            idx_min=('idx', 'first'),
            idx_max=('idx', 'last')
        ).reset_index()

        # order the chunks by idx_min
        chunks = chunks.sort_values('idx_min').reset_index(drop=True)

        #print(chunks)

        # check if there is no movement chunk between two movement chunks that is shorter than 200 ms
        for i in range(1, len(chunks)-1):
            if chunks.loc[i, 'anno_values'] == 'no movement' and chunks.loc[i-1, 'anno_values'] == 'movement' and chunks.loc[i+1, 'anno_values'] == 'movement':
                if chunks.loc[i, 'time_ms_max'] - chunks.loc[i, 'time_ms_min'] < 200:
                    print('found a chunk of no movement between two movement chunks that is shorter than 200 ms')
                    # change the chunk to movement in the original df
                    anno_df.loc[chunks.loc[i, 'idx_min']:chunks.loc[i, 'idx_max'], 'anno_values'] = 'movement'

        # Calculate new chunks
        anno_df['chunk'] = (anno_df['anno_values'] != anno_df['anno_values'].shift()).cumsum()
        anno_df['idx'] = anno_df.index

        chunks = anno_df.groupby(['anno_values', 'chunk']).agg(
            time_ms_min=('time_ms', 'first'),
            time_ms_max=('time_ms', 'last'),
            idx_min=('idx', 'first'),
            idx_max=('idx', 'last')
        ).reset_index()

        # order the chunks by idx_min
        chunks = chunks.sort_values('idx_min').reset_index(drop=True)

        # now check if there is any chunk of movement in between no movement chunks that is shorter than 250 ms
        for i in range(1, len(chunks)-1):
            if chunks.loc[i, 'anno_values'] == 'movement' and chunks.loc[i-1, 'anno_values'] == 'no movement' and chunks.loc[i+1, 'anno_values'] == 'no movement':
                if chunks.loc[i, 'time_ms_max'] - chunks.loc[i, 'time_ms_min'] < 200:
                    print('found a chunk of movement between two no movement chunks that is shorter than 250 ms')
                    # change the chunk to no movement in the original df
                    anno_df.loc[chunks.loc[i, 'idx_min']:chunks.loc[i, 'idx_max'], 'anno_values'] = 'no movement'

        
        # now in the anno_df in anno_values, take the first value of movement and the last value of movement, and anything in between is movement too
        # if there is any movement in anno_values
        if 'movement' in anno_df['anno_values'].unique():
            # get the first and last index of movement
            first_idx = anno_df[anno_df['anno_values'] == 'movement'].index[0]
            last_idx = anno_df[anno_df['anno_values'] == 'movement'].index[-1]

            # change all values in between to movement
            anno_df.loc[first_idx:last_idx, 'anno_values'] = 'movement'

        # Calculate new chunks
        anno_df['chunk'] = (anno_df['anno_values'] != anno_df['anno_values'].shift()).cumsum()
        anno_df['idx'] = anno_df.index

        chunks = anno_df.groupby(['anno_values', 'chunk']).agg(
            time_ms_min=('time_ms', 'first'),
            time_ms_max=('time_ms', 'last'),
            idx_min=('idx', 'first'),
            idx_max=('idx', 'last')
        ).reset_index()

        # order the chunks by idx_min
        chunks = chunks.sort_values('idx_min').reset_index(drop=True)

        print(chunks)

        # Rewrite "no movement" in anno_values to "nomovement"
        chunks['anno_values'] = chunks['anno_values'].apply(
            lambda x: 'nomovement' if x == 'no movement' else x
        )

        # Add elanID to chunks
        chunks['elanID']  = str(filename + '_ELAN_tiers.eaf')

        # Write to the text file
        with open(txtfile, 'a') as f:
            for _, row in chunks.iterrows():
                # Write each line formatted without extra metadata
                f.write(
                    f"Anno_R2\t{row['time_ms_min']}\t{row['time_ms_max']}\t{row['anno_values']}\t{row['elanID']}\n"
                )

 

e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_11_p1_time_annotations_threshold_0_8.csv
   anno_values  chunk  time_ms_min  time_ms_max  idx_min  idx_max
0  no movement      1            0         3196        0     3196
e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_12_p1_time_annotations_threshold_0_8.csv
   anno_values  chunk  time_ms_min  time_ms_max  idx_min  idx_max
0  no movement      1            0         3724        0     3724
e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampling_dataset_0_1_13_p1_time_annotations_threshold_0_8.csv
   anno_values  chunk  time_ms_min  time_ms_max  idx_min  idx_max
0  no movement      1            0         3484        0     3484
e:\FLESH_ContinuousBodilyEffort\04_TS_movementAnnotation\ML_Annotations\arms_annotations_threshold_0_8\sampli

Now we want to merge the files of interest

In [142]:
# open AutoAnno_arms.txt and ManualAnno_arms.txt
# check which files are in the ManualAnno_arms.txt but not in AutoAnno_arms.txt
# get rows that contains these files from both txt files into one txt file IA_arms

toi = ['arms', 'head', 'upper', 'lower'] # mov missing for now

for tier in toi:
    print('working on ' + tier)
    txtfile_auto = interfolder + 'AutoAnno_' + tier + '.txt'
    txtfile_manual_r1 = interfolder + 'R3_Manual_' + tier + '.txt'
    txtfile_manual_r3 = interfolder + 'R1_Manual_' + tier + '.txt'

    comp1 = 'R1' # change here who you want to compare
    comp2 = 'R3'   # with whom

    #read the files
    r1_anno = pd.read_csv(txtfile_manual_r1, sep='\t', header=None) # change here who you want to compare
    r2_anno = pd.read_csv(txtfile_manual_r3, sep='\t', header=None)    # with whom

    #get the files that are in manual_arms but not in auto_arms
    files_to_check_r1 = r1_anno[4].unique()
    files_to_check_r2 = r2_anno[4].unique()

    #create a list that contains files that are in both lists
    files_to_check = list(set(files_to_check_r1) & set(files_to_check_r2))

    # put away those that have 0_1_34, 0_2_42, and 0_2_66 in them - these are faulty trials in pose2sim so we dont have auto anno for them
    #files_to_check = [x for x in files_to_check if '0_1_34' not in x and '0_2_42' not in x and '0_2_66' not in x]

    # get the rows that contain these files from auto_arms
    rows_auto = r1_anno[r1_anno[4].isin(files_to_check)]
    rows_manual = r2_anno[r2_anno[4].isin(files_to_check)]

    #concat rows_to_check_auto with manual_arms
    concat_rows = pd.concat([r1_anno, r2_anno])

    #save the rows to a new txt file
    txtfile_IA_arms = interfolder + 'IA_' + comp1 + '_' + comp2 + '_' + tier + '.txt'

    with open(txtfile_IA_arms, 'w') as f:
        for index, row in concat_rows.iterrows():
            f.write(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}\t{row[4]}\n")

working on arms
working on head
working on upper
working on lower


Interrater agreement

Auto+R1

kappa for arms: 0.8
kappa for head: 0.61
lower body: 0.71
upper body: 0.69

R1+R2
arms: 0.85
head: 0.67
lower body: 0.74
upper body: 0.7

